In [1]:
import numpy as np
import torch
import torch.nn as nn

##### nn.Linear layer

In [10]:
torch.manual_seed(2023)
input = torch.randn(2, 4)
print(input.size())
print(input)

torch.Size([2, 4])
tensor([[-1.2075,  0.5493, -0.3856,  0.6910],
        [-0.7424,  0.1570,  0.0721,  1.1055]])
